# Seq2Seq

이름 : Jungwoo Kim

기수 : 11th

작성자 : 10 기 신재우

In [1]:
# colab 환경에서 학습을 진행하실 분들은 구글드라이브를 연동해주세요
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 시작 전 실행할 것들

In [2]:
%pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 49.1 MB/s eta 0:00:00


In [3]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
import pandas as pd
import numpy as np
import tqdm
import spacy
import torchtext
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import random
import os
from ast import literal_eval
import torchtext.vocab as vocab
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

### 경로 설정

In [4]:
path_to_folder = "/content/drive/MyDrive/[0225]RNN + Transformers_JungwooKim"

In [5]:
path_train = os.path.join(path_to_folder, "Dataset/train_dataset_Tokenized.csv")
path_val = os.path.join(path_to_folder, "Dataset/val_dataset_Tokenized.csv")
path_test = os.path.join(path_to_folder, "Dataset/test_dataset_Tokenized.csv")

train_dataset = pd.read_csv(path_train, index_col = 0)
val_dataset = pd.read_csv(path_val, index_col = 0)
test_dataset = pd.read_csv(path_test, index_col = 0)

# Vocabulary

In [6]:
train_dataset['en_tokens'] = train_dataset['en_tokens'].apply(literal_eval)
train_dataset['kr_tokens'] = train_dataset['kr_tokens'].apply(literal_eval)

val_dataset['en_tokens'] = val_dataset['en_tokens'].apply(literal_eval)
val_dataset['kr_tokens'] = val_dataset['kr_tokens'].apply(literal_eval)

test_dataset['en_tokens'] = test_dataset['en_tokens'].apply(literal_eval)
test_dataset['kr_tokens'] = test_dataset['kr_tokens'].apply(literal_eval)

train_dict = train_dataset.to_dict(orient = 'records')
val_dict = val_dataset.to_dict(orient = 'records')
test_dict = test_dataset.to_dict(orient = 'records')

In [7]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"
sos_token = "<sos>"
eos_token = "<eos>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

In [8]:
def yield_tokens(data, token_field):
    for item in data:
        yield item[token_field]

en_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "en_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

kr_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "kr_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

en_vocab.set_default_index(en_vocab[unk_token])
kr_vocab.set_default_index(kr_vocab[unk_token])

In [9]:
vocab_size_en = len(en_vocab)
vocab_size_kr = len(kr_vocab)
print("English Vocabulary Length : ", vocab_size_en)
print("Korean Vocabulary Length : ", vocab_size_kr)

English Vocabulary Length :  9772
Korean Vocabulary Length :  10442


# 문제 2 (Vocabulary Matching)

토큰화된 단어들을 Vocabulary 과 매칭 시켜서 **벡터**로 바꿔준 뒤에 **텐서**로 바꿔주는 단계까지 포함되어야 합니다!

힌트 :

- $\texttt{en_vocab.lookup_indices}$, 혹은 $\texttt{kr_vocab.lookup_indices}$
- 한줄로 표현이 가능합니다!
- en_indices 와 kr_indices 는 리스트이며, 텐서화 된 것들을 각 토큰마다 리스트로 가져야 합니다!

In [10]:
en_indices = []
for word in train_dataset['en_tokens']:
  en_indices.append(en_vocab.lookup_indices(word))

train_dataset['en_indices'] = en_indices

kr_indices = []
for word in train_dataset['kr_tokens']:
  kr_indices.append(kr_vocab.lookup_indices(word))

train_dataset['kr_indices'] = kr_indices

In [11]:
en_indices = []
for word in val_dataset['en_tokens']:
  en_indices.append(en_vocab.lookup_indices(word))

val_dataset['en_indices'] = en_indices

kr_indices = []
for word in val_dataset['kr_tokens']:
  kr_indices.append(kr_vocab.lookup_indices(word))

val_dataset['kr_indices'] = kr_indices

In [12]:
en_indices = []
for word in test_dataset['en_tokens']:
  en_indices.append(en_vocab.lookup_indices(word))

test_dataset['en_indices'] = en_indices

kr_indices = []
for word in test_dataset['kr_tokens']:
  kr_indices.append(kr_vocab.lookup_indices(word))

test_dataset['kr_indices'] = kr_indices

# Data Loaders

In [13]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):

        return len(self.dataframe)

    def __getitem__(self, idx):

        item = self.dataframe.iloc[idx]
        en_indices = item['en_indices']
        kr_indices = item['kr_indices']

        return en_indices, kr_indices

In [21]:
batch_no = 80

def collate_fn(batch):
    en_indices, kr_indices = zip(*batch)

    # 리스트의 각 요소를 텐서로 변환
    en_indices_tensors = [torch.tensor(seq) for seq in en_indices]
    kr_indices_tensors = [torch.tensor(seq) for seq in kr_indices]

    # 변환된 텐서 리스트를 사용하여 패딩 적용
    en_indices_padded = pad_sequence(en_indices_tensors, batch_first=True, padding_value=0)
    kr_indices_padded = pad_sequence(kr_indices_tensors, batch_first=True, padding_value=0)

    return en_indices_padded, kr_indices_padded

train_custom_dataset = CustomDataset(train_dataset[['en_indices', 'kr_indices']])
val_custom_dataset = CustomDataset(val_dataset[['en_indices', 'kr_indices']])
test_custom_dataset = CustomDataset(test_dataset[['en_indices', 'kr_indices']])

train_loader = DataLoader(train_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)

In [22]:
def get_collate_fn(pad_index):

    def collate_fn(batch):
        batch_en_ids = [example["en_indices"] for example in batch]
        batch_de_ids = [example["de_indices"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }

        return batch

    return collate_fn

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )

    return data_loader

### 잘 매칭이 되었는지 확인해보기!

In [23]:
for i, (en_indices, kr_indices) in enumerate(train_loader):
    print(f"Batch {i+1}")
    print(f"Shape of en_indices: {en_indices.shape}")
    print(f"Shape of kr_indices: {kr_indices.shape}")

    print("Sample en_indices batch:", en_indices[:1])
    print("Sample kr_indices batch:", kr_indices[:1])

    if i == 1:
        break

Batch 1
Shape of en_indices: torch.Size([80, 28])
Shape of kr_indices: torch.Size([80, 41])
Sample en_indices batch: tensor([[   2, 3636,   11,    5,  510,   19,    5, 2248,   76,    8,  594,   14,
         1354, 1090,  137, 3902,    8,  212,    0,   23, 1357,    4,    3,    0,
            0,    0,    0,    0]])
Sample kr_indices batch: tensor([[   2, 4557,   20, 5662,   24,   11, 1885,  583,    9,  477,  122,    5,
           11, 1906, 6973,  546,    8,  308,    5,   12,   15,    4,    3,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]])
Batch 2
Shape of en_indices: torch.Size([80, 30])
Shape of kr_indices: torch.Size([80, 31])
Sample en_indices batch: tensor([[   2,    5,  878,   90,   25, 2686, 3499,   24, 2966, 3169,   22,    5,
          504,  796,    4,    3,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]])
Sample kr_indices batch: tensor([[   2,   10,  165,

# 문제 3 (Modules without Attention!)

기본적인 Seq2seq 모듈들은 Teacher Forcing 까지 포함해서 진행해주시길 바랍니다.

In [25]:
class Encoder(nn.Module):
  """
  Encoder 내에서는 Input Language 의 Vocab 크기, Embedding 크기, Hidden 크기가 들어갑니다.

  Embedding 과정에서 Input 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.

  Input : [Batch_size, Input Sequences Max Size]
  Output : [Batch_size, Input Sequences Max Size, Hidden Size]
  Hidden : [1, Batch_size, Hidden Size]

  힌트 :
  - Batch 가 앞에 있기 때문에 GRU 는 batch_first = True 라는 파라미터를 설정해줘야 합니다.
  - GRU 에서는 Output, Hidden Vector, 총 2개를 뱉어줍니다.
  - GRU 는 nn.GRU, Embedder 는 nn.Embedding 를 사용해주면 됩니다.
  """
  def __init__(self, input_size, embedding_size, hidden_size):
    super(Encoder, self).__init__()
    # 임베딩 층 초기화
    self.embedding = nn.Embedding(input_size, embedding_size)
    # GRU 층 초기화, batch_first=True로 설정하여 입력 텐서의 첫 번째 차원이 배치 크기가 되도록 함
    self.gru = nn.GRU(embedding_size, hidden_size, batch_first=True)

  def forward(self, input_seq):
    # 입력 시퀀스를 임베딩, 결과는 [Batch_size, Input Sequences Max Size, Embedding Size]
    embedded = self.embedding(input_seq)
    # 임베딩된 시퀀스를 GRU에 통과시킴, GRU 출력은 [Batch_size, Input Sequences Max Size, Hidden Size]
    output, hidden = self.gru(embedded)

    return output, hidden

In [32]:
class Decoder(nn.Module):
  """
  Decoder 내에서는 Hidden 크기, Embedding Decoder 크기 (Embedding Encoder 크기와 동일), Output Language 의 Vocab 의 크기가 들어갑니다.

  Embedding 과정에서 Output 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.
  Linear 내에서는 Hidden 크기 --> Output 크기 로 진행됩니다.

  Teacher Forcing 은 디코더를 구성하는 코드에서는 고려를 안해도 되며, 나중에 Seq2Seq 모듈에서 진행해주시길 바랍니다.

  Input 과 Hidden 크기의 경우 Seq2Seq 모듈 내에서 차원을 맞춰주시면 됩니다.
  """
  def __init__(self, hidden_size, embedding_size, output_size):
    super(Decoder, self).__init__()
    # 임베딩 층 초기화
    self.embedding = nn.Embedding(output_size, embedding_size)
    # GRU 층 초기화
    self.gru = nn.GRU(embedding_size, hidden_size, batch_first=True)
    # 선형 층 초기화
    self.fc = nn.Linear(hidden_size, output_size)
    self.output_size = output_size

  def forward(self, input, hidden):
    # 입력을 임베딩, 결과는 [Batch_size, 1, Embedding Size]로 변환 (디코더에서는 한 번에 하나의 단어만 처리)
    input = input.unsqueeze(1)
    embedded = self.embedding(input)
    # 임베딩된 입력을 GRU에 통과
    output, hidden = self.gru(embedded, hidden)
    # GRU의 출력을 선형 층에 통과
    preds = self.fc(output.squeeze(1))
    return preds, hidden





In [33]:
class Seq2Seq(nn.Module):
  """
  Seq2Seq 내에서는 Encoder, Decoder, Device 가 들어갑니다.

  Init 단계에서는 정말로 할 것이 크게 없으며, forward 단계에서 for loop 과 if 조건문들을 활용해서 Teacher Forcing 을 적용시켜주면 됩니다.

  Input : [Batch_size, Input Sequences Max Size]
  Target : [Batch_size, Target Sequences Max Size]
  Output : [Batch_size, Input Sequences Max Size, Input Language Vocab Size]

  힌트 :
  - outputs 란 0 으로 채워진 텐서를 하나 만들며 디코더의 t-번째 아웃풀을 outputs 의 t-번째 칸에 넣으면서 for loop 을 진행해주시면 됩니다.
  - Target 의 첫번째 요소 (:, 0) 는 항상 <sos> 입니다! 이것을 디코더의 첫 Input 으로 넣어준 다음에 for loop 을 진행해주시면 됩니다.
  - random.random() < teacher_forcing_ratio 를 통해서 조건문을 만들 수가 있습니다!
  - Teacher Force 가 적용이 안될 때는디코더의 아웃풋에는 모든 토큰들에 대한 점수가 나오게 됩니다.
    이것들 중에서 가장 최댓값만 중요하기에 argmax 를 이용해서 최댓값의 인덱스를 다음 Decoder 의 Input 으로 넣어주면 됩니다.
  - squeeze 와 unsqueeze 를 적극적으로 활용해서 차원을 조정해주시면 됩니다!
  """
  def __init__(self, encoder, decoder, device):
      super(Seq2Seq, self).__init__()
      self.encoder = encoder
      self.decoder = decoder
      self.device = device

  def forward(self, input, target, teacher_forcing_ratio=0.5):
      batch_size = target.shape[0]
      target_len = target.shape[1]
      target_vocab_size = self.decoder.output_size

      # 모든 디코더 출력을 저장하기 위한 텐서 초기화
      outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)

      # 인코더의 마지막 hidden state를 디코더의 첫 hidden state로 사용
      _, hidden = self.encoder(input)

      # 디코더의 첫 입력은 <sos> 토큰
      decoder_input = target[:, 0]

      for t in range(1, target_len):
          # 디코더에 입력을 전달하고 다음 출력과 hidden state를 받음
          output, hidden = self.decoder(decoder_input, hidden)
          outputs[:, t] = output
          # 티처 포싱 사용: 다음 입력으로 실제 목표를 사용할지, 아니면 이전 출력을 사용할지 결정
          teacher_force = random.random() < teacher_forcing_ratio
          # 가장 높은 확률을 가진 토큰 얻기
          top1 = output.argmax(1)
          # 입력을 다음 시간 단계를 위해 업데이트: teacher forcing이면 다음 타겟을, 아니면 최대 확률 토큰을 사용
          decoder_input = target[:, t] if teacher_force else top1

      return outputs

# Training

In [34]:
input_dim = len(en_vocab)
output_dim = len(kr_vocab)
encoder_embedding_dim = 512
decoder_embedding_dim = 512
hidden_dim = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim)
decoder = Decoder(hidden_dim, decoder_embedding_dim, output_dim)
model = Seq2Seq(encoder, decoder, device).to(device)

### 모델 확인 + Initialize 시키기

In [35]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 30,502,090 trainable parameters


### 하이퍼 파라미터 설정!

In [36]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

## Training 코드!

In [37]:
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0

    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss

    model.eval()
    epoch_loss = 0
    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()

    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

Epoch: 1, Train Loss : 2.7898564319610597, Val Loss: 2.9754299436296736
Epoch: 2, Train Loss : 2.4688101857503257, Val Loss: 2.8984543028331937
Epoch: 3, Train Loss : 2.2949284454981487, Val Loss: 2.771564903713408
Epoch: 4, Train Loss : 2.1290878947575886, Val Loss: 2.851076046625773
Epoch: 5, Train Loss : 1.9763104155858358, Val Loss: 2.7917011995164174
Epoch: 6, Train Loss : 1.8126893733342488, Val Loss: 2.846702496210734
Epoch: 7, Train Loss : 1.6513143994013468, Val Loss: 2.878590610292223
Epoch: 8, Train Loss : 1.5097451108296713, Val Loss: 2.9685493499513655
Epoch: 9, Train Loss : 1.368552811940511, Val Loss: 3.0902778837415905
Epoch: 10, Train Loss : 1.2724007142384848, Val Loss: 3.105707637847416


파라미터 저장

In [38]:
path_to_save = os.path.join(path_to_folder, "model_without_attention.pt")
torch.save(model.state_dict(), path_to_save)

### 적용시켜보기

In [39]:
test_batch_en, test_batch_kr = next(iter(test_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()
model.eval()
with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

['<sos>', 'if', 'so', ',', 'send', 'the', 'two', 'types', 'of', 'correct', 'test', 'request', 'as', 'the', 'file', 'i', 'attached', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<sos>', '그렇', '다면', '나', '가', '첨부', '하', 'ㄴ', '파일', '처럼', ',', '두', '종류', '의', '올바르', 'ㄴ', '시험', '요청서', '를', '보내주', '세요', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '아', '능하', '다면', '파일', '시', 'ㄴ', '파일', '하', '다시', '파일', '다시', '내', '을', '내', 'ㅂ니다', 'ㅂ니다', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']


\<unk\> 는 저희 Vocabulary 에는 존재하지 않는 토큰들을 의미합니다.

아무리 epoch 를 늘리거나 무엇을 해보려고 해도 완전히 좋은 결과가 나오지는 않을 것입니다. 해당 이유는 Attention 이 적용이 안되었기 때문이며, 다음과 같이 Attention 을 적용시켜서 한번 진행하면 결과가 다르다는 것을 알 수가 있습니다.

# 문제 4 (Modules WITH Attention)

만약에 메모리가 다 찼다면 해당 Run 을 Reconnect 시켜서 위의 Output 만 남긴 채로 제출하시면 됩니다.

**이렇게 진행하게 된다면 당연히 문제 3 이전의 코드들은 다시 돌려야 됩니다!!**

In [56]:
class Encoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size):
    super(Encoder, self).__init__()
    # 임베딩 층 초기화
    self.embedding = nn.Embedding(input_size, embedding_size)
    # GRU 층 초기화, batch_first=True로 설정하여 입력 텐서의 첫 번째 차원이 배치 크기가 되도록 함
    self.gru = nn.GRU(embedding_size, hidden_size, batch_first=True)

  def forward(self, input_seq):
    # 입력 시퀀스를 임베딩, 결과는 [Batch_size, Input Sequences Max Size, Embedding Size]
    embedded = self.embedding(input_seq)
    # 임베딩된 시퀀스를 GRU에 통과시킴, GRU 출력은 [Batch_size, Input Sequences Max Size, Hidden Size]
    output, hidden = self.gru(embedded)

    return output, hidden

In [95]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.softmax = nn.Softmax(dim=1)  # Attention 가중치에 대한 소프트맥스 적용 차원

    def forward(self, hidden, encoder_outputs):
        # hidden 상태 조정: [1, Batch_size, Hidden_size] -> [Batch_size, Hidden_size]
        hidden = hidden.squeeze(0)  # [Batch_size, Hidden_size]

        # encoder_outputs와 hidden의 내적 계산을 위해 hidden 차원 변환
        hidden = hidden.unsqueeze(2)  # [Batch_size, Hidden_size, 1]

        # 배치 행렬 곱 실행
        attention_scores = torch.bmm(encoder_outputs, hidden).squeeze(2)  # [Batch_size, Seq_length]
        attention_weights = self.softmax(attention_scores).unsqueeze(1)  # [Batch_size, 1, Seq_length]

        # 컨텍스트 벡터 계산
        context_vector = torch.bmm(attention_weights, encoder_outputs)  # [Batch_size, 1, Hidden_size]

        return context_vector


In [96]:
class Decoder(nn.Module):
  """
  Attention Value 를 Attention 모듈을 통해서 구했기 때문에 Concatenate, Tangent Hyperbolic 등의 것들이 여기에서 적용이 됩니다.

  힌트 :
  - 모든 것을 진행하기 전에 앞서서 먼저 GRU 를 지나고 나서 Output 을 활용해야 합니다.
  - Attention 모듈도 사용해야 하기 때문에 해당 모듈을 현 모듈에서 정의도 해줘야 합니다.
  - Concatenate 의 경우 torch.cat((x, y), dim = 1) 형태로 사용하면 됩니다.
  - W_c 와 v_t 를 곱해줄 때 W_c 는 가중치 행렬이며, 이것은 그저 nn.Linear 형태로 표현이 가능합니다.
  """
  def __init__(self, hidden_size, embedding_size, output_size):
      super(Decoder, self).__init__()
      self.hidden_size = hidden_size
      self.embedding = nn.Embedding(output_size, embedding_size)
      self.gru = nn.GRU(embedding_size + hidden_size, hidden_size, batch_first=True)
      self.fc = nn.Linear(hidden_size * 2, output_size)
      self.attention = Attention(hidden_size)
      self.output_size = output_size  # Ensure output_size is defined for later use

  def forward(self, input, hidden, encoder_outputs):
      # input: [Batch_size] -> Expected to be the current token indices
      input = input.unsqueeze(1)  # [Batch_size, 1] for embedding
      embedded = self.embedding(input)  # [Batch_size, 1, Embedding_size]

      # Calculate attention weights and apply to encoder outputs
      context_vector = self.attention(hidden, encoder_outputs)  # hidden 대신에 정확한 차원을 가진 텐서 사용

      # Concatenate embedded input token and context vector
      gru_input = torch.cat((embedded, context_vector), dim=2)  # [Batch_size, 1, Embedding_size + Hidden_size]

      # Pass through GRU
      output, hidden = self.gru(gru_input, hidden)

      # Concatenate the output from GRU with context vector, and pass through the fully connected layer
      output = output.squeeze(1)  # [Batch_size, Hidden_size]
      context_vector = context_vector.squeeze(1)  # [Batch_size, Hidden_size]
      output = torch.cat((output, context_vector), dim=1)  # [Batch_size, Hidden_size * 2]
      output = self.fc(output)  # [Batch_size, Output_size]

      return output, hidden



In [97]:
class Seq2Seq(nn.Module):
  """
  이전과 Seq2Seq 부분은 크게 달라지는게 없습니다!
  단지 디코더가 Input 을 하나 더 받기 때문에 조심해주시길 바랍니다.
  """
  def __init__(self, encoder, decoder):
      super(Seq2Seq, self).__init__()
      self.encoder = encoder
      self.decoder = decoder

  def forward(self, input, target, teacher_forcing_ratio=0.5):
      batch_size = target.size(0)
      target_len = target.size(1)
      target_vocab_size = self.decoder.output_size

      # 모든 디코더 출력을 저장하기 위한 텐서를 초기화합니다.
      outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(input.device)

      # 인코더를 통과시켜 컨텍스트 벡터를 얻습니다.
      encoder_outputs, hidden = self.encoder(input)

      # 디코더의 첫 입력은 항상 <sos> 토큰입니다.
      decoder_input = target[:, 0]

      # 디코더를 시퀀스 길이만큼 반복합니다.
      for t in range(1, target_len):
          # 디코더를 한 시간 단계로 전달합니다.
          decoder_output, hidden = self.decoder(decoder_input, hidden, encoder_outputs)

          # 디코더 출력을 저장합니다.
          outputs[:, t] = decoder_output

          # 티처 포싱: 다음 입력으로 실제 타겟을 사용할지, 아니면 이전 출력을 사용할지 결정합니다.
          teacher_force = random.random() < teacher_forcing_ratio

          # 다음 디코더 입력을 결정합니다.
          top1 = decoder_output.argmax(1)
          decoder_input = target[:, t] if teacher_force else top1

      return outputs

# Training

In [98]:
input_dim = len(en_vocab)
output_dim = len(kr_vocab)
encoder_embedding_dim = 512
decoder_embedding_dim = 512
hidden_dim = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim)
decoder = Decoder(hidden_dim, decoder_embedding_dim, output_dim)
model = Seq2Seq(encoder, decoder).to(device)

### 모델 확인 + Initialize 시키기

In [99]:
model.apply(init_weights)
print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 44,340,426 trainable parameters


### 하이퍼 파라미터 설정!

In [100]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

### 학습 시키기!

In [101]:
for epoch in range(n_epochs):

    model.train()
    epoch_loss = 0

    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()
    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

Epoch: 1, Train Loss : 2.7517386843363445, Val Loss: 3.036093651302277
Epoch: 2, Train Loss : 2.3580064776738485, Val Loss: 2.907860763489254
Epoch: 3, Train Loss : 2.1085519803365074, Val Loss: 2.693421973122491
Epoch: 4, Train Loss : 1.8955144764582317, Val Loss: 2.751033718623812
Epoch: 5, Train Loss : 1.6936092929840088, Val Loss: 2.738518612725394
Epoch: 6, Train Loss : 1.5476713139216105, Val Loss: 2.9107996206434947
Epoch: 7, Train Loss : 1.4053480631510418, Val Loss: 2.8484508272201294
Epoch: 8, Train Loss : 1.3025057338078816, Val Loss: 2.908921858621022
Epoch: 9, Train Loss : 1.2159946835835775, Val Loss: 2.8983208149198503
Epoch: 10, Train Loss : 1.1344917809168498, Val Loss: 3.0632919205559626


### 파라미터 저장

In [102]:
path_to_save = os.path.join(path_to_folder, "model_WITH_attention.pt")
torch.save(model.state_dict(), path_to_save)

### 적용시켜보기

In [103]:
test_batch_en, test_batch_kr = next(iter(train_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()

model.eval()

with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

['<sos>', 'i', "'m", 'afraid', 'it', 'was', 'my', 'fault', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<sos>', '그', '게', '저', '의', '잘못', '때문', '이', '아니', 'ㄴ가', '걱정', '이', '되', '어요', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '게', '저', '의', '잘못', '때문', '이', 'ㄹ까', '었', '.', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']


아마 결과가 온전히 잘 나오지는 않을 겁입니다. 해당 이유는 단순히 파라미터 개수가 적기 때문이며, 잘 나오기 위해서는 이보다 훨씬 늘려야 합니다. 원하신다면 Hidden_size, Embedding_size 를 늘려서 진행하는 것도 가능하긴 합니다!

과제 하시느라 정말로 고생하셨습니다!